[Training](#Training)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, classification_report, RocCurveDisplay, roc_auc_score, roc_curve, ConfusionMatrixDisplay, \
    precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
import os

In [ ]:
path = '../Data/df_clean_null.pkl'
df = pd.read_pickle(path)

In [ ]:
feature_names = ['_RFHLTH', '_HCVU651', '_RFHYPE5', '_RFCHOL', '_ASTHMS1', '_DRDXAR1', '_RACEGR3', '_AGE_G', '_BMI5CAT', '_EDUCAG', '_INCOMG', '_SMOKER3',\
    '_RFDRHV5', '_PACAT1', '_RFSEAT2', '_FLSHOT6', '_PNEUMO2', '_AIDTST3', 'CHCCOPD1', 'ADDEPEV2', 'CHCKIDNY', 'DIABETE3', 'SEX', 'MARITAL', 'DRADVISE']
target_name = ['_MICHD']
weights_name = ['_LLCPWT'] 

In [ ]:
df = df[feature_names + target_name + weights_name]
df

In [ ]:
for column in ['_RFHYPE5', '_RFCHOL', '_RFDRHV5']:
    df[column] = df[column].apply(lambda x: 1.0 if x == 2.0 else 2.0 if x == 1.0 else x)

In [ ]:
for column in ['_FLSHOT6', '_PNEUMO2']:
    df[column] = df[column].cat.add_categories([65.0])
    df[column] = df[column].fillna(65.0)

In [ ]:
df['DIABETE3'] = df['DIABETE3'].map({1.0:1.0, 2.0:1.0, 3.0:2.0, 4.0:2.0, 7.0:7.0, 9.0:9.0}).astype('category')

In [ ]:
df['_MICHD'] = df['_MICHD'].apply(lambda x: 0.0 if x == 2 else x)

In [ ]:
def imputer(dataframe, category_value_tofill=None, columns_drop=None, columns_mode=None, columns_median=None):
    '''Imputes missing values to the input dataframe.
    
       Parameters
       ----------
       dataframe: Pandas dataframe
           dataframe with which to impute missing values.
       
       category_value_tofill: int, float, or string
           Value to used to fill missing values in categorical features.
       
       columns_drop: list-like
           List of columns to drop.
       
       columns_mode: list-like
           List of numeric columns to impute with the mode.
           
       columns_median: list-like
           List of numeric columns to impute with the mean.
    '''
    #Fill null values in categorical features with value_null
    if category_value_tofill != None:
        for column in dataframe.select_dtypes(include='category').columns:
            if any(dataframe[column].isnull()):
                dataframe[column] = dataframe[column].cat.add_categories([category_value_tofill])
                dataframe[column] = dataframe[column].fillna(value=category_value_tofill)
            
    #Droping columns, imputing with mode, and imputing with median.
    if columns_drop != None:
        dataframe = dataframe.drop(columns=columns_drop)
    if columns_mode != None:
        dataframe = dataframe.fillna(dataframe[columns_mode].mode().iloc[0, :])
    if columns_median != None:
        dataframe = dataframe.fillna(dataframe[columns_median].median())
    
    return dataframe

In [ ]:
df = imputer(df, 999.0)

In [ ]:
trans_list = ['Good_Health', 'Health_Insurance', 'Hypertension', 'High_Cholesterol', 'Asthma_Status', 'Arthritis', 'Race', 'Age_Cat', 'BMI_Cat', 'Education_Level', 'Income_Level', \
    'Smoker_Status', 'Heavy_Drinker', 'Physical_Activity', 'Seatbelt', 'Flu_Shot', 'Pneumonia_Vaccine', 'HIV', 'Bronchitis', 'Depression', 'Kidney_Disease', 'Diabetes', 'SEX', \
        'Marital Status', 'Sodium', 'Heart_Disease', 'Sample_Weights']

In [ ]:
trans_dict = dict(zip(feature_names + target_name + weights_name, trans_list))
trans_dict

In [ ]:
df = df.rename(columns=trans_dict)
df

In [ ]:
df = df.drop(df[df[trans_dict[target_name[0]]] == 999.0].index)

In [ ]:
cols_to_one_hot = [column for column in df.select_dtypes(include='category').columns if column not in ['Heart_Disease']]

column_trans = ColumnTransformer([('categorical', OneHotEncoder(sparse=False), cols_to_one_hot)], remainder='passthrough')
column_trans.fit(df)
column_names_trans = np.concatenate([column_trans.named_transformers_['categorical'].get_feature_names_out(), np.array([trans_dict[target_name[0]], trans_dict[weights_name[0]]])])

In [ ]:
df = pd.DataFrame(column_trans.transform(df), columns=column_names_trans)
df

In [ ]:
with open('../Data/modeling_df.pkl', 'wb') as f:
    pickle.dump(df, f)
    pickle.dump(trans_dict, f)

## Training

In [ ]:
with open('../Data/modeling_df.pkl', 'rb') as f:
    df = pickle.load(f)
    
y = df['Heart_Disease']
w = df['Sample_Weights']
X = df.drop(columns=[y.name, w.name])

In [ ]:
class LogisticRegressionCustom(LogisticRegression):
    def __init__(self, max_iter=100, class_weight=None, threshold=None):
        super().__init__(max_iter=max_iter, class_weight=class_weight)
        self.threshold = threshold
    
    def predict(self, X):
        if self.threshold != None:
            return (super().predict_proba(X)[:, 1] >= self.threshold).astype(int)
        else:
            return super().predict(X)

In [ ]:
def plot_logistic_feature_importance(coefficients, feature_names, number):
    coef_enum = enumerate(coefficients)
    coef_enum_sorted_top = sorted(coef_enum, key=lambda x: abs(x[1]), reverse=True)[:number]
    
    indices = list(list(zip(*coef_enum_sorted_top))[0])
    names = feature_names[indices]
    top_coef = np.array(list(zip(*coef_enum_sorted_top))[1])
    
    neg_pos = np.where(top_coef < 0)[0]
    pos_pos = np.where(top_coef >= 0)[0]
    neg_coef = top_coef[neg_pos]
    pos_coef = top_coef[pos_pos]
    neg_names = names[neg_pos]
    pos_names = names[pos_pos]
    
    _, ax = plt.subplots()
    
    ax.bar(neg_pos, np.abs(neg_coef), color='r', label='Negative')
    ax.bar(pos_pos, np.abs(pos_coef), color='b', label='Positive')
    ax.set_xticks(range(0, len(names)))
    ax.set_xticklabels(names, rotation=90)
    ax.set_title('Top Features by Absolute Value of Model Coefficient')
    ax.set_ylabel('Feature Coefficient')
    ax.set_xlabel('Features')
    
    ax.legend()
    plt.show()

In [ ]:
def find_threshold(y_true, y_proba, set_tpr, sample_weight=None):
    _, tpr_list, threshold_list = roc_curve(y_true, y_proba, sample_weight=sample_weight)
    for tpr, threshold in zip(tpr_list, threshold_list):
        if (tpr >= set_tpr):
            return tpr, threshold

In [ ]:
def my_scorer(y_test, y_pred, y_pred_proba=None, sample_weight=None, model_name=None, output=True):
    prfs_scores = precision_recall_fscore_support(y_test, y_pred, sample_weight=sample_weight)
    roc_auc_score_ = roc_auc_score(y_test, y_pred, sample_weight=sample_weight)
    scores = *prfs_scores, roc_auc_score_
    
    cf_display = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, sample_weight=sample_weight, normalize = 'true')
    if output == True:
        if type(model_name) != type(None):
            plt.title(model_name)
        plt.show()
        print()
        
    if output == True:
        print(f'Classification Report: {model_name if type(model_name) != type(None) else ""}')
    class_report = classification_report(y_test, y_pred, sample_weight=sample_weight)
    if output == True:
        print(class_report)
        print()
    
    if type(y_pred_proba) != type(None):
        roc_display = RocCurveDisplay.from_predictions(y_test, y_pred_proba, sample_weight=sample_weight)
        if output == True:
            if model_name != None:
                plt.title(model_name)
            plt.show()
    else:
        roc_display = None
        
    return scores, (cf_display, class_report, roc_display)

In [ ]:
def my_pipeline(ml_algo, *data, model_name=None, algo_params={}, resampler=None, split_params={}, output=True):
    if len(data) not in [2, 3]:
        print(f'Invalid length for "data": {len(data)}.')
        return
    
    results = {}
    
    split_result = train_test_split(*data, **split_params)
    if len(split_result) == 4:
        X_train, X_test, y_train, y_test = split_result
        w_train = None
        w_test = None
    elif len(split_result) == 6:
        X_train, X_test, y_train, y_test, w_train, w_test = split_result
    else:
        print('Invalid length for "split_result".')
        return
        
    if type(resampler) != type(None):
        X_train, y_train = resampler().fit_resample(X_train, y_train)
    
    model = ml_algo(**algo_params)
    model.fit(X_train, y_train, sample_weight=w_train)
    results['model_info'] = model_name, model
    
    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)
    if hasattr(model, 'predict_proba'):
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba_train = None
        y_pred_proba = None
        
    results['predictions_train'] = y_pred_train, y_pred_proba_train
    results['predictions'] =y_pred, y_pred_proba
    
    results['scores_train'], results['displays_train'] = my_scorer(y_train, y_pred_train, y_pred_proba_train, sample_weight=w_train, model_name=model_name + ' (train)', output=output)
    results['scores'], results['displays'] = my_scorer(y_test, y_pred, y_pred_proba, sample_weight=w_test, model_name=model_name, output=output)
    
    return results


In [ ]:
def grid_cv_scorer(ml_algo, scoring, *data, model_name=None, param_grid={}, resampler=None, output=True):    
    if len(data) == 2:
        X, y = tuple(data)
        w = None
    elif len(data) == 3:
        X, y, w = tuple(data)
    else:
        print('Invalid length for "data".')
        return
    
    if type(resampler) != type(None):
        model = Pipeline([('Resampling', resampler()), (model_name, ml_algo())])
    else:
        model = Pipeline([(model_name, ml_algo())])
        
    keys = [k for k in param_grid.keys()]
    for key in keys:
        param_grid[model_name + '__' + key] = param_grid.pop(key)
    
    clf = GridSearchCV(model, param_grid=param_grid, scoring=scoring, refit=False)
    clf.fit(X, y, CVSearch_Test__sample_weight=w)
    cv_scores = clf.cv_results_
    print('Printing cv_scores keys:')
    print(cv_scores.keys())
    print('Printing cv_scores:')
    print(cv_scores)
    if output == True:
        if type(model_name) != type(None):
            print(model_name + ' cv_scores:')
        else:
            print('cv_scores:')
        print(cv_scores)
        print()
    
    dict_scores = {}
    #dict_scores['estimator'] = cv_scores['estimator']
    for score in scoring:
        #scores_ = cv_scores['test_' + score]
        mean_array = cv_scores['mean_test_' + score]
        std_array = cv_scores['std_test_' + score]
        
        dict_scores[score]= dict(zip(['mean', 'std'], [mean_array, std_array]))
        
        if output == True:
            print(score + ' mean: ', mean_array)
            print(score + ' std: ', std_array)
            print()
    if print == True:
        print()
    
    if type(model_name) != type(None):
        return {model_name:dict_scores}
    else:
        return dict_scores

In [ ]:
def cv_scorer(ml_algo, scoring, *data, model_name=None, algo_params={}, resampler=None, output=True):    
    if len(data) == 2:
        X, y = tuple(data)
        w = None
    elif len(data) == 3:
        X, y, w = tuple(data)
    else:
        print('Invalid length for "data".')
        return
    
    if type(resampler) != type(None):
        model = Pipeline([('Resampling', resampler()), (model_name, ml_algo(**algo_params))])
    else:
        model = Pipeline([(model_name, ml_algo(**algo_params))])
    
    cv_scores = cross_validate(model, X, y, scoring=scoring, fit_params={model_name + '__sample_weight':w}, return_estimator=True)
    if output == True:
        if type(model_name) != type(None):
            print(model_name + ' cv_scores:')
        else:
            print('cv_scores:')
        print(cv_scores)
        print()
    
    dict_scores = {}
    dict_scores['estimator'] = cv_scores['estimator']
    for score in scoring:
        scores_ = cv_scores['test_' + score]
        mean_ = cv_scores['test_' + score].mean()
        std_ = cv_scores['test_' + score].std()
        
        dict_scores[score]= dict(zip(['scores', 'mean', 'std'], [scores_, mean_, std_]))
        
        if output == True:
            print(score + ' mean: ' + f'{mean_:0.2f}')
            print(score + ' std: ' + f'{std_:0.4f}')
            print()
    if print == True:
        print()
    
    if type(model_name) != type(None):
        return {model_name:dict_scores}
    else:
        return dict_scores

In [ ]:
def plot_feature_importance(importances, labels, title=None, num=20):
    num = num * (-1) - 1
    indices = importances.argsort()[:num:-1]
    plt.bar(x=labels[indices], height=importances[indices])
    plt.xticks(rotation=90)
    if type(title) != type(None):
        plt.title(title)
    plt.show()

In [ ]:
precision_score_pos = make_scorer(precision_score, pos_label=1)
precision_score_neg = make_scorer(precision_score, pos_label=0)

recall_score_pos = make_scorer(recall_score, pos_label=1)
recall_score_neg = make_scorer(recall_score, pos_label=0)

f1_score_pos = make_scorer(f1_score, pos_label=1)
f1_score_neg = make_scorer(f1_score, pos_label=0)

roc_auc_scorer = make_scorer(roc_auc_score)

In [ ]:
grid_cv_results = grid_cv_scorer(LogisticRegressionCustom, {'precision':precision_score_pos, 'recall':recall_score_pos}, X, y, model_name='CVSearch_Test', param_grid={'max_iter':[500, 600]}, \
    resampler=RandomOverSampler)

In [ ]:
grid_cv_results

In [ ]:
algo_list = {'Logistic Regression':LogisticRegressionCustom, 'Random Forest':RandomForestClassifier}
algo_params_list = [dict(max_iter=500), dict(n_estimators=100)]
imbalance_strategy_list = {'Basic':None, 'Class Weight':None, 'Random Oversmpling':RandomOverSampler, 'Smote':SMOTE, 'Random Undersampler':RandomUnderSampler}
scores = {'precision_pos':precision_score_pos, 'precision_neg':precision_score_neg, 'recall_pos':recall_score_pos, 'recall_neg':recall_score_neg, \
    'f1_pos':f1_score_pos, 'f1_neg':f1_score_neg, 'roc_auc':roc_auc_scorer}

cv_results = {}

for algo_name, algo_params in zip(algo_list, algo_params_list):
    for resampler_name, resampler in imbalance_strategy_list.items():
        if resampler_name == 'Basic':
            data = X, y, w
        else:
            data = X, y
        if resampler_name == "Class Weight":
            algo_params['class_weight'] = 'balanced'
            
        cv_results.update(cv_scorer(algo_list[algo_name], scores, *data, model_name=algo_name+', '+resampler_name, algo_params=algo_params, resampler=resampler))
        
        with open('../Data/modeling_results.pkl', 'wb') as f:
            pickle.dump(cv_results, f)


In [ ]:
algo_list = {'Random Forest':RandomForestClassifier}
algo_params_list = [dict(n_estimators=100)]
imbalance_strategy_list = {'Class Weight':None, 'Random Oversmpling':RandomOverSampler, 'Smote':SMOTE, 'Random Undersampler':RandomUnderSampler}
scores = {'precision_pos':precision_score_pos, 'precision_neg':precision_score_neg, 'recall_pos':recall_score_pos, 'recall_neg':recall_score_neg, \
    'f1_pos':f1_score_pos, 'f1_neg':f1_score_neg, 'roc_auc':roc_auc_scorer}

for algo_name, algo_params in zip(algo_list, algo_params_list):
    for resampler_name, resampler in imbalance_strategy_list.items():
        if resampler_name == 'Basic':
            data = X, y, w
        else:
            data = X, y
        if resampler_name == "Class Weight":
            algo_params['class_weight'] = 'balanced'
            
        cv_results.update(cv_scorer(algo_list[algo_name], scores, *data, model_name=algo_name+', '+resampler_name, algo_params=algo_params, resampler=resampler))
        
        with open('../Data/modeling_results.pkl', 'wb') as f:
            pickle.dump(cv_results, f)

In [ ]:
with open('../Data/modeling_results.pkl', 'rb') as f:
    cv_results = pickle.load(f)

In [ ]:
cv_results.keys()

In [ ]:
cv_results.pop('Random Forest, Basic')
cv_results.keys()

In [ ]:
columns = []
for score in scores:
    columns.append(score + ' mean')
    columns.append(score + ' std')
    
data = []
index = []
for model_name in cv_results:
    index.append(model_name)
    row = []
    for score in cv_results[model_name]:
        row.append(cv_results[model_name][score]['mean'])
        row.append(cv_results[model_name][score]['std'])
    data.append(row)
columns, index, data

In [ ]:
results_df = pd.DataFrame(data, index=index, columns=columns)
results_df

In [ ]:
with open('../Data/modeling_results.pkl', 'wb') as f:
    pickle.dump(cv_results, f)